# Processing Large Files

We now have a list of Long Island zipcodes, and we can use it to make sure we are only looking at Long Island data. There's an awful lot of data, though - 18GB, in 54 separate files. We could read each file separately into Pandas data frames, and then filter them to the Long Island zipcodes. But, there is a better way! We can use Polars, a new dataframe library built on Rust for speed and performance. We rely on some neat features along the way to write less code and get more for it. 

In [8]:
import polars as pl

We use the header file description from the FEC to figure out the columns and their order. This lets us see, for instance, that STATE is column 10 and ZIP_CODE is column 11. 

In [9]:
header_columns = """CMTE_ID,AMNDT_IND,RPT_TP,TRANSACTION_PGI,IMAGE_NUM,TRANSACTION_TP,ENTITY_TP,NAME,CITY,STATE,ZIP_CODE,EMPLOYER,OCCUPATION,TRANSACTION_DT,TRANSACTION_AMT,OTHER_ID,TRAN_ID,FILE_NUM,MEMO_CD,MEMO_TEXT,SUB_ID
""".split(",")
for i, c in enumerate(header_columns):
    print(i+1, c)

1 CMTE_ID
2 AMNDT_IND
3 RPT_TP
4 TRANSACTION_PGI
5 IMAGE_NUM
6 TRANSACTION_TP
7 ENTITY_TP
8 NAME
9 CITY
10 STATE
11 ZIP_CODE
12 EMPLOYER
13 OCCUPATION
14 TRANSACTION_DT
15 TRANSACTION_AMT
16 OTHER_ID
17 TRAN_ID
18 FILE_NUM
19 MEMO_CD
20 MEMO_TEXT
21 SUB_ID



We load the list of zipcodes we created in our last notebook. 

In [10]:
import geopandas as gpd
li_zip_codes = gpd.read_file("../references/long_island_zipcodes.geojson")
zip_codes = li_zip_codes.postalcode.unique().tolist()

Unfortunately, we can't just apply our header columns to the data. As we might expect from such a large set of data, some of it is a mess. Surprises I found while trying to process the data: 

- Non-US zipcodes like 'W11 3LN'. This is a postal code in England (!!!!!)
- Bad tranaction_dt 'LINCOLN'
- Records missing some of the required columns

That last one is the most serious because, if we provide a list of columns to polars, it forces every record to have the same number of fields and fails when that's not true. But, we can refer to the columns by their positions instead, treat every column as utf8, and then apply filters to the data to make sure we only get the zipcodes we want. We also tell polars to scan the data in chunks, rather than try to read the entire file into RAM. This code is unbelievably performant - it scanned 18GB of data from my machine in just over a minute. 

In [11]:
fec_df = pl.scan_csv(
    "../data/raw/by_date/*.txt", 
    separator='|', 
    has_header=False, 
    infer_schema_length=0,
    ignore_errors=True,
    missing_utf8_is_empty_string=True,
).filter(pl.col("column_10") == "NY").filter(pl.col("column_11").str.slice(0,5).is_in(zip_codes)).collect(streaming=True)

STREAMING CHUNK SIZE: 2380 rows


Now that we have a subset of the data, we can apply our header column to it so we can get the names we want to work with, which were taken from here:

https://www.fec.gov/campaign-finance-data/contributions-individuals-file-description/

In [12]:
fec_df.columns = header_columns

We can then write the file to parquet, which will let us use it with Pandas in our next notebook. 

In [13]:
fec_df.write_parquet("../data/processed/individual_2020_li.parquet")